In [ ]:
%pip install azure-cognitiveservices-speech --quiet
%pip install openai==1.30.1 --quiet
%pip install sounddevice --quiet
%pip install azure-ai-formrecognizer azure-cognitiveservices-speech azure-identity --quiet

In [20]:
import azure.cognitiveservices.speech as speechsdk

speech_key, service_region = "<api_key>", "swedencentral"

def recognize_from_microphone():
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    speech_config.speech_recognition_language="en-US"

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

recognize_from_microphone()

Speak into your microphone.
Recognized: Hello, my name is Thomas. What is on that picture?


In [16]:
from openai import AzureOpenAI

openai_key = "<api_key>"
openai_endpoint = "https://ut-oai-sweden.openai.azure.com/"
openai_deployment = "whisper"
    
client = AzureOpenAI(
    api_key=openai_key,
    api_version="2024-02-01",
    azure_endpoint = openai_endpoint
)

audio_test_file = "./helloThere.m4a"

result = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=openai_deployment
)

print(result)

Transcription(text='Hello there!')


In [21]:
'''
  For more samples please visit https://github.com/Azure-Samples/cognitive-services-speech-sdk 
'''

import azure.cognitiveservices.speech as speechsdk

# Creates an instance of a speech config with specified subscription key and service region.
speech_key = "f07276fad5414743b35c407402f0e57d"
service_region = "swedencentral"

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
# Note: the voice setting will not overwrite the voice element in input SSML.
speech_config.speech_synthesis_voice_name = "en-GB-RyanNeural"

text = "Hi, this is Ryan"

# use the default speaker as audio output.
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

result = speech_synthesizer.speak_text_async(text).get()
# Check result
if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized for text [{}]".format(text))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))



Speech synthesized for text [Hi, this is Ryan]
